# 김기범 전치리 코드 테스트

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('../data/train.csv')

train_df.shape

(52530, 8)

# 1. add predefined news category

In [3]:
train_df['input_text'][:10]

0         유튜브 내달 2일까지 크리에이터 지원 공간 운영
1            어버이날 막따가 흐려저…남부지방 여튼 황사
2            어버이날 맑다가 흐려져…남부지방 옅은 황사
3        내년부터 국가RD 평가 때 논문건수는 반영 않는다
4    김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
5     회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간
6     회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간
7         야외서 생방송 하세요…액션캠 전용 요금제 잇따라
8         월드컵 태극전사 16강 전초기지 레오강 입성종합
9                         미세먼지 속 출근길
Name: input_text, dtype: object

In [4]:
train_df['input_text'][:10] + " [SEP] " + train_df['predefined_news_category'][:10]

0         유튜브 내달 2일까지 크리에이터 지원 공간 운영 [SEP] IT과학
1            어버이날 막따가 흐려저…남부지방 여튼 황사 [SEP] 생활문화
2            어버이날 맑다가 흐려져…남부지방 옅은 황사 [SEP] 생활문화
3        내년부터 국가RD 평가 때 논문건수는 반영 않는다 [SEP] IT과학
4    김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것 [SEP] IT과학
5     회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간 [SEP] 생활문화
6     회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간 [SEP] 생활문화
7         야외서 생방송 하세요…액션캠 전용 요금제 잇따라 [SEP] IT과학
8          월드컵 태극전사 16강 전초기지 레오강 입성종합 [SEP] 스포츠
9                           미세먼지 속 출근길 [SEP] 사회
dtype: object

# 2. AI HUB 뉴스 기사 데이터 가져오기

In [5]:
from glob import glob

In [6]:
ai_hub_017 = glob('../data/ai_hub_017/*.json')
ai_hub_017

['../data/ai_hub_017/TS_unanswerable.json',
 '../data/ai_hub_017/TS_span_inference.json',
 '../data/ai_hub_017/VS_unanswerable.json',
 '../data/ai_hub_017/VS_span_inference.json',
 '../data/ai_hub_017/TS_text_entailment.json',
 '../data/ai_hub_017/VS_span_extraction.json',
 '../data/ai_hub_017/TS_span_extraction.json',
 '../data/ai_hub_017/VS_text_entailment.json']

In [14]:
import json

df = pd.DataFrame()

for path in ai_hub_017:
    try:
        with open(path, 'r') as f:
            js = json.loads(f.read())
    except Exception as e:
        print(e)

    new_df = pd.json_normalize(js['data'])

    df = pd.concat([df, new_df], axis=0)

df.shape

Invalid control character at: line 250348 column 279 (char 18754979)


(158020, 8)

In [15]:
df.columns

Index(['doc_id', 'doc_title', 'doc_source', 'doc_published', 'created',
       'paragraphs', 'doc_class.class', 'doc_class.code'],
      dtype='object')

In [16]:
df.head()

,doc_id,doc_title,doc_source,doc_published,created,paragraphs,doc_class.class,doc_class.code
0,01100201.20210225173715002,"“시진핑엔 선물, 바이든엔 모욕”… 유럽의회서 막힌 中·EU투자협정",국민일보,20210225,20211217144411,[{'context': '중국과 유럽연합(EU)이 체결하기로 합의한 포괄적투자협정(...,None,국제
1,01100201.20210225092831001,김경수 분노…“내가 문 대통령께 반발? 참 희한한 일”,국민일보,20210225,20211217144411,[{'context': '문재인 대통령의 최측근으로 꼽히는 김경수 경남지사는 25일...,None,정치
2,01100201.20210225111004001,오늘부터 사라진 네이버 ‘실검’…빈자리엔 날씨와 주가,국민일보,20210225,20211217144411,[{'context': '국내 최대 포털 네이버에서 25일 ‘실시간 급상승 검색어(...,None,IT과학
3,01100201.20210225040953001,"경남도, 신혼 반값 임대주택 공급 확대",국민일보,20210225,20211217144411,"[{'context': '경남도와 김해시, 경남개발공사, 인제대학교가 청년·신혼부부...",None,사회
4,01100201.20210225161345001,작년 3분기 일자리 늘었다는데… 2030 일자리만 15만개 감소,국민일보,20210225,20211217144411,[{'context': '지난해 3분기 임금근로 일자리가 지난해 같은 기간보다 36...,None,경제


In [17]:
df['doc_class.code'].unique()

array(['국제', '정치', 'IT과학', '사회', '경제', '문화', '스포츠', '지역', '기타'],
      dtype=object)

In [18]:
use_classes = ['정치', '경제', '사회', '문화', '스포츠', 'IT과학', '국제']

In [48]:
df2 = df[(df['doc_class.code'] != '지역') & (df['doc_class.code'] != '기타')][['doc_id', 'doc_title', 'doc_class.code']]
df2.shape

(111998, 3)

In [49]:
df2.drop_duplicates()

df2.shape

(111998, 3)

In [50]:
df2['doc_class.code'].unique()

array(['국제', '정치', 'IT과학', '사회', '경제', '문화', '스포츠'], dtype=object)

In [51]:
df2['doc_class.code'] = df2['doc_class.code'].replace('국제', '세계')
df2['doc_class.code'] = df2['doc_class.code'].replace('문화', '생활문화')

df2['doc_class.code'].unique()

array(['세계', '정치', 'IT과학', '사회', '경제', '생활문화', '스포츠'], dtype=object)

In [52]:
train_df.columns, df2.columns

(Index(['ID', 'input_text', 'label_text', 'target', 'predefined_news_category',
        'annotations', 'url', 'date'],
       dtype='object'),
 Index(['doc_id', 'doc_title', 'doc_class.code'], dtype='object'))

In [55]:
label2num = {
    label: i for i, label in enumerate(['정치', '경제', '사회', '생활문화', '세계', 'IT과학', '스포츠'])
}
label2num

{'정치': 0, '경제': 1, '사회': 2, '생활문화': 3, '세계': 4, 'IT과학': 5, '스포츠': 6}

In [56]:
df2.columns = ['ID', 'input_text', 'label_text']
df2['target'] = df2['label_text'].apply(lambda x: label2num[x])

df2.columns

Index(['ID', 'input_text', 'label_text', 'target'], dtype='object')

In [59]:
print(pd.concat([train_df, df2], axis=0).shape)
print(pd.concat([train_df, df2], axis=0).drop_duplicates(subset=['input_text']).shape)
print(pd.concat([train_df, df2], axis=0).drop_duplicates(subset=['input_text', 'target']).shape)

(164528, 8)
(161537, 8)
(162937, 8)


In [58]:
# df2.to_csv('../data/ai_hub_017.csv', index=False)

# 3. AI HUB 낚시 기사성 뉴스 데이터

In [61]:
ai_hub_146_val_path_list = glob("../data/ai_hub_146_val/*/*.json")
len(ai_hub_146_val_path_list)

46903

In [83]:
id = []
category = []
title = []

for path in ai_hub_146_val_path_list:
    new_df = pd.read_json(path)

    id.append(new_df.loc['newsID']['sourceDataInfo'])
    category.append(new_df.loc['newsCategory']['sourceDataInfo'])
    title.append(new_df.loc['newsTitle']['sourceDataInfo'])

df = pd.DataFrame()
df['ID'] = id
df['input_text'] = category
df['label_text'] = title

df.shape

(46903, 3)

In [86]:
df.columns = ['ID', 'label_text', 'input_text']
df['label_text'].unique()

array(['IT&과학', '세계', '정치', '사회', '생활&문화', '경제'], dtype=object)

In [89]:
df['label_text'] = df['label_text'].replace(r'&', '', regex=True)
df['label_text'].unique()

array(['IT과학', '세계', '정치', '사회', '생활문화', '경제'], dtype=object)

In [90]:
df2['target'] = df2['label_text'].apply(lambda x: label2num[x])
df2.head()

,ID,input_text,label_text,target
0,01100201.20210225173715002,"“시진핑엔 선물, 바이든엔 모욕”… 유럽의회서 막힌 中·EU투자협정",세계,4
1,01100201.20210225092831001,김경수 분노…“내가 문 대통령께 반발? 참 희한한 일”,정치,0
2,01100201.20210225111004001,오늘부터 사라진 네이버 ‘실검’…빈자리엔 날씨와 주가,IT과학,5
3,01100201.20210225040953001,"경남도, 신혼 반값 임대주택 공급 확대",사회,2
4,01100201.20210225161345001,작년 3분기 일자리 늘었다는데… 2030 일자리만 15만개 감소,경제,1


In [91]:
# df2.to_csv('../data/ai_hub_146_val.csv', index=False)